In [31]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/KVector_Merge/sponge'
testcase = 'books_join_small_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/KVector_Merge/sponge/fb_join_small_1/outputs/results/run2', '/home/chesetti/Repos/KVector_Merge/sponge/fb_join_small_1/outputs/results/run1', '/home/chesetti/Repos/KVector_Merge/sponge/fb_join_small_1/outputs/results/run0']


In [32]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_index_build_duration_ns', 'result.inner_index_size',
       'result.outer_disk_fetch', 'result.outer_index_build_duration',
       'result.outer_index_size', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.type', 'spec.inner_table',
       'spec.key_size', 'spec.key_type', 'spec.load_sstable_in_mem',
       'spec.name', 'spec.num_threads', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk',
       'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [33]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name   btree_1page  btree_2page  btree_4page   pgm1024    pgm128  \
spec.common_key                                                              
10                  18805756     17290803     16291594  17292097  13838773   
20                  12085448     10336360      9421767  12972155   9379264   
30                  10216891      7938393      7132877  11466507   8057949   
40                   8750128      6840844      6057255  10943163   7593219   
50                   8167564      6215304      5339063  10428474   7093966   
60                   7665669      5752662      4881415  10121175   6747487   
70                   7185393      5460966      4543458   9743861   6464019   
80                   7036071      5263280      4417902   9348218   6222014   
90                   6539365      5033676      4228280   9135145   6151043   
100                  6391858      4867042      4080282   8882520   5898021   

spec.algo_name     pgm256    pgm512     pgm64        sj       s

In [34]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

       indexes  memory_bytes
0  btree_1page       70032.0
1  btree_2page       37072.0
2  btree_4page       20592.0
3      pgm1024         120.0
4       pgm128        1128.0
5       pgm256         328.0
6       pgm512         168.0
7        pgm64        4064.0


In [35]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))

In [36]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
print(inner_index_fetch)

spec.algo_name   btree_1page  btree_2page  btree_4page  pgm1024  pgm128  \
spec.common_key                                                           
10                      3896         1948          974     3890    3897   
20                      3896         1948          974     3890    3896   
30                      3894         1948          974     3889    3897   
40                      3887         1948          974     3883    3882   
50                      3874         1948          974     3865    3860   
60                      3848         1946          974     3824    3826   
70                      3789         1946          974     3771    3773   
80                      3731         1942          974     3741    3725   
90                      3647         1943          974     3652    3661   
100                     3597         1935          974     3589    3604   

spec.algo_name   pgm256  pgm512  pgm64    sj   sj2  
spec.common_key                               

In [37]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 30])
print(result_checksum.loc[result_checksum['spec.common_key'] == 30]['result.checksum'].unique())

common_key: 10 checksum: ['B1AE56F68540653EBFB1A9DCFFDEFFAE'] OK
common_key: 20 checksum: ['2685906541017A324208E4280A9635B4'] OK
common_key: 30 checksum: ['2108CDE32BFD93412363270BE08D00BC'] OK
common_key: 40 checksum: ['6441AB3EE55D1AF00C40E1EBFFC10FF1'] OK
common_key: 50 checksum: ['A8A7F78FFE9E90347263CDF232D94301'] OK
common_key: 60 checksum: ['499EBC47F3CB77C543481910789034CC'] OK
common_key: 70 checksum: ['15B70C8C0AA017A29672DE65D2BEB6AE'] OK
common_key: 80 checksum: ['0EBE2F95AC58F2B18B619673288AC5E8'] OK
common_key: 90 checksum: ['46A2523D6FD3C6253DDCFC691988BD3C'] OK
common_key: 100 checksum: ['CF78C259FC3B13F7D80482AFC5E18489'] OK
     spec.common_key spec.algo_name                   result.checksum
39                30    btree_1page  2108CDE32BFD93412363270BE08D00BC
139               30    btree_1page  2108CDE32BFD93412363270BE08D00BC
239               30    btree_1page  2108CDE32BFD93412363270BE08D00BC
4                 30    btree_2page  2108CDE32BFD93412363270BE08D00BC